In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from os import path

import requests
import datetime
import pathlib
import apimoex
import time

from tqdm.notebook import tqdm

FMP_API_KEY = "uILltAaGY2ms0reL0RVtgtALlh2BbYH5"
PATH = "C:\Program Files (x86)\chromedriver.exe" # driver path
parent_dir = r"C:\Users\Никита\Андан\Project\data" # root for data 

## Рабочий парсер MOEX

In [8]:
# |sector|country|ticker|...|
companies = pd.read_csv('companies_list.csv')

In [16]:
def parse_tickers(companies, fmp_api_key):
    board = 'TQBR' # здесь важно поресерчить, чем отличаются
    today = str(datetime.date.today())

    
    companies_ru = companies[companies.country == 'RU']
    with requests.Session() as session:
        for indx, row in tqdm(companies_ru.iterrows(), desc = 'Processing russian stock', total = companies_ru.shape[0]):
        
            data = apimoex.get_board_history(session, row.ticker, board=board)
            
            if data == []:
                print('Empty set')
            df = pd.DataFrame(data)

            if row.sector == 'Renewable Energy':
                df.to_csv(path.join(parent_dir, 'renewable_energy', f'{row.ticker}_RU.csv'))
            elif row.sector == 'Healthcare':
                df.to_csv(path.join(parent_dir, 'healthcare_services', f'{row.ticker}_RU.csv'))
            elif row.sector == 'Financial Services':
                df.to_csv(path.join(parent_dir, 'fintech', f'{row.ticker}_RU.csv'))
            elif row.sector == 'Industrials':
                df.to_csv(path.join(parent_dir, 'industrial_goods', f'{row.ticker}_RU.csv'))
    
    companies_usa = companies[companies.country == 'USA']
    for indx, row in tqdm(companies_usa.iterrows(), desc = 'Processing USA stock', total = companies_usa.shape[0]):
        try:
            link = f"https://financialmodelingprep.com/api/v3/historical-chart/4hour/{row.ticker}?to={today}&apikey={fmp_api_key}"
            df = pd.DataFrame(requests.get(link).json())
        except ValueError:
            print('Stopped for 25 hours')
            time.sleep(25*60*60) # оставить на 25 часов

            link = f"https://financialmodelingprep.com/api/v3/historical-chart/4hour/{row.ticker}?to=2024-04-07&apikey={fmp_api_key}"
            df = pd.DataFrame(requests.get(link).json())
            

        if row.sector == 'Renewable Energy':
            df.to_csv(path.join(parent_dir, 'renewable_energy', f'{row.ticker}_USA.csv'))
        elif row.sector == 'Healthcare':
            df.to_csv(path.join(parent_dir, 'healthcare_services', f'{row.ticker}_USA.csv'))
        elif row.sector == 'Financial Services':
            df.to_csv(path.join(parent_dir, 'fintech', f'{row.ticker}_USA.csv'))
        elif row.sector == 'Industrials':
                df.to_csv(path.join(parent_dir, 'industrial_goods', f'{row.ticker}_USA.csv'))
            

            

In [ ]:
parse_tickers(companies, FMP_API_KEY)

Processing russian stock:   0%|          | 0/9 [00:00<?, ?it/s]

Processing USA stock:   0%|          | 0/340 [00:00<?, ?it/s]

Stopped for 25 hours


## Парсер FMP

In [11]:
ticker = 'IMPP'
date = str(datetime.date.today())
APIKEY = "uILltAaGY2ms0reL0RVtgtALlh2BbYH5"

link = f"https://financialmodelingprep.com/api/v3/historical-chart/4hour/{ticker}?to={date}&apikey={APIKEY}"
response = requests.get(link)

In [13]:
pd.DataFrame(response.json()).head()

,date,open,low,high,close,volume
0,2024-04-11 12:00:00,3.7350,3.6801,3.740,3.7199,42527
1,2024-04-10 12:00:00,3.4850,3.4600,3.600,3.5800,266530
2,2024-04-09 12:00:00,3.4250,3.3800,3.480,3.4300,224928
3,2024-04-08 12:00:00,3.7700,3.6000,3.835,3.6603,512603
4,2024-04-05 12:00:00,3.6324,3.4398,3.700,3.5500,953191


## Нерабочий парсер

In [18]:
def extract_list_of_companies(link, driver):
    driver.get(link)

    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((
            By.CLASS_NAME, 'link link--blue table-child--middle align-top')))

    browser.quit()
    return elements
    

In [19]:
driver = webdriver.Chrome(PATH)

elements = extract_list_of_companies(link, driver)

TimeoutException: Message: 
